## The third Lab-assignment (07/22/2022 11:59'AM' - 07/26/2022 11:59PM, 50 points in total)

The purpose of this exercise is to understand users' information needs, then collect data from different sources for analysis.

Question 1 (10 points). Fomulate your domain problem: Describe an interesting research question (or practical question) you have in mind, what kind of data should be collected to answer the question(s)? How many data needed for the analysis? The detail steps for collecting and save the data. 

In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:
QUESTION:
What city in texas has the most population in 2021 and what is the increase in population in those cities every decade?

ANSWER:

To answer this question I have collected an tabular data from the reddit database:

Source of data : https://www.reddit.com/

It is recommended to collected user-generated posts from the Reddit social media using Application Programming Interface 
(API) web scraping technology.
Collection of Reddit posts and comments from multiple bounding boxes in parallel and compliance with Reddit API
usage limits. The time period of collectioned data is from last 2 years.

I scaped the data from Reddit posts need to consist of at least the words: sub reddit as 'food' and keyword as meat. 'post_id', 'title', 'selftext',
'url', 'author', 'score', 'publish_date', 'num_of_comments', 'permalink', 'flair'

 
Then I scraped and downloaded as csv file in my local folder as tabular format names raddit.csv. it represents top 1000 
obesity bases posts and comments from Reddit social media.

After Extracting the information I have saved the results into a dataframe. Now, using the data stored in the dataframe, and 
I can perform analysis.

Using all the information present in the datafram I can make conclusions such as on what factor influencing for obesity from their food behavior. 

'''

Question 2 (30 points). Collect your data to answer the research problem: Write python code to collect 1000 data samples you discussed above.

In [8]:
# You code here (Please add comments in the code):

import numpy as np
import requests
import json
import csv
import time
import datetime
import os


def get_pushshift_data(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title=' + str(query) + '&size=1000&after=' + str(
        after) + '&before=' + str(before) + '&subreddit=' + str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']


def collect_sub_data(subm):
    sub_data = list()  # list to store data points
    title = subm['title']
    url = subm['url']
    try:
        # if flair is available then get it, else set 'NaN'
        flair = subm['link_flair_text']
    except KeyError:
        flair = 'NaN'
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']

    try:
        # if selftext is available then get it, else set it empty
        selftext = subm['selftext']
        list_of_empty_markers = ['[removed]', '[deleted]']
        # many times selftext would be removed or deleted, if thats the case then set it empty
        if selftext in list_of_empty_markers:
            selftext = ''
    except:
        selftext = ''
    created = datetime.datetime.fromtimestamp(subm['created_utc'])  # 1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']

    sub_data.append((sub_id, title, selftext, url, author, score, created, numComms, permalink, flair))
    sub_stats[sub_id] = sub_data


def write_subs_to_file(filename):
    upload_count = 0
    if os.path.exists(filename):
        keep_header = False
    else:
        keep_header = True

    with open(filename, 'a', newline='',encoding="utf-8") as file:
        a = csv.writer(file, delimiter=',')
        headers = ['post_id', 'title', 'selftext', 'url', 'author', 'score', 'publish_date', 'num_of_comments',
                   'permalink', 'flair']
        if keep_header:
            a.writerow(headers)
        for sub in sub_stats:
            a.writerow(sub_stats[sub][0])
            upload_count += 1
        # print(str(upload_count) + ' submissions have been uploaded')


if __name__ == '__main__':
    # Download reddit posts from sub_reddit with keywords given by key_word

    sub_reddit = 'food'
    key_word = 'meat'

    output_filename = 'reddit.csv'
    # search all the posts from start_date to end_date overall
    start_date = datetime.datetime(2020, 7, 23, 0)
    end_date = datetime.datetime(2022, 7, 23, 0)

    # in each itration get reddit posts for one day, to avoid getting blocked by server
    one_day = datetime.timedelta(hours=24)
    after_date = start_date
    after = str(int(after_date.timestamp()))
    before_date = start_date + one_day
    before = str(int(before_date.timestamp()))

    while after_date < end_date:
        print('-' * 80)
        print(after_date, ' -> ', before_date)
        print('-' * 80)

        sub_count = 0
        sub_stats = {}

        data = get_pushshift_data(key_word, after, before, sub_reddit)

        max_count = 200
        count = 0
        while len(data) > 0 and count < max_count:
            print('count ', count)
            for submission in data:
                collect_sub_data(submission)
                sub_count += 1

            print(len(data))
            print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
            after = data[-1]['created_utc']
            data = get_pushshift_data(key_word, after, before, sub_reddit)
            # print(data)
            # print(data['data'][0]['author'])
            count = count + 1

        # keep saving data collected in each iteration
        write_subs_to_file(output_filename)

        # move to next day
        after_date += one_day
        after = str(int(after_date.timestamp()))
        before_date += one_day
        before = str(int(before_date.timestamp()))

        # randomly sleep before starting next iteration
        time.sleep(np.random.randint(1, 3))

--------------------------------------------------------------------------------
2020-07-23 00:00:00  ->  2020-07-24 00:00:00
--------------------------------------------------------------------------------
https://api.pushshift.io/reddit/search/submission/?title=meat&size=1000&after=1595480400&before=1595566800&subreddit=food
count  0
1
2020-07-23 16:20:21
https://api.pushshift.io/reddit/search/submission/?title=meat&size=1000&after=1595539221&before=1595566800&subreddit=food
--------------------------------------------------------------------------------
2020-07-24 00:00:00  ->  2020-07-25 00:00:00
--------------------------------------------------------------------------------
https://api.pushshift.io/reddit/search/submission/?title=meat&size=1000&after=1595566800&before=1595653200&subreddit=food
count  0
3
2020-07-24 23:44:23
https://api.pushshift.io/reddit/search/submission/?title=meat&size=1000&after=1595652263&before=1595653200&subreddit=food
------------------------------------

In [4]:
import pandas as pd

data  = pd.read_csv(r'C:\UNT\Courses\INFO 5502\reddit.csv', encoding = 'unicode_escape') 
data

,post_id,title,selftext,url,author,score,publish_date,num_of_comments,permalink,flair
0,kox4mm,"Musaca greceasca vegan, reteta sanatoasa de sl...",NaN,https://sanatoase.blogspot.com/2021/01/Musaca....,Retete_Sanatoase,1,1/2/2021 7:01,2,/r/food/comments/kox4mm/musaca_greceasca_vegan...,NaN
1,kp6zk3,A vegan in germany ^.^ [homemade],NaN,https://i.redd.it/8khxdz4jtz861.jpg,Calypso_lume,1,1/2/2021 16:11,2,/r/food/comments/kp6zk3/a_vegan_in_germany_hom...,NaN
2,kqiv0a,[Homemade] Vegan Tacos and Margaritas,NaN,https://i.redd.it/nizzorsazd961.jpg,WookieSuave,1,1/4/2021 15:49,2,/r/food/comments/kqiv0a/homemade_vegan_tacos_a...,NaN
3,kql40e,[homemade] best vegan stew recipe!!,NaN,https://youtu.be/LvmmtZUuZ8U,icebergleticia,1,1/4/2021 17:37,2,/r/food/comments/kql40e/homemade_best_vegan_st...,NaN
4,kqw6mq,[Homemade] Homemade buns with vegan chickpea b...,NaN,https://i.imgur.com/vgpUdNp.jpg,stjimmyjos,1,1/5/2021 4:54,2,/r/food/comments/kqw6mq/homemade_homemade_buns...,NaN
...,...,...,...,...,...,...,...,...,...,...
2580,u7f9a7,[I ate] Elk meat pasty,NaN,https://i.imgur.com/OlYb5aE.jpg,lo-key-glass,1,4/19/2022 15:54,1,/r/food/comments/u7f9a7/i_ate_elk_meat_pasty/,NaN
2581,u8hi3q,[homemade] veal meat ð (last Saturday),NaN,https://www.reddit.com/gallery/u8hi3q,THE_VOLK87,1,4/21/2022 1:57,1,/r/food/comments/u8hi3q/homemade_veal_meat_las...,NaN
2582,u8hisw,veal meat [homemade] last Saturday,NaN,https://www.reddit.com/gallery/u8hisw,THE_VOLK87,1,4/21/2022 1:58,1,/r/food/comments/u8hisw/veal_meat_homemade_las...,NaN
2583,u90yob,[Homemade]This is my take on a Japanese Gyudon...,NaN,https://i.redd.it/dgmaw5l3zyu81.jpg,Irizarryeats,1,4/21/2022 18:49,1,/r/food/comments/u90yob/homemadethis_is_my_tak...,NaN


Question 3 (10 points). Understand the data quality: Search a second hand dataset (any dataset) from kaggle or other websites. Describe the data quality problem of the dataset and explain your strtegy to clean the data.

In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):
'''

Please write you answer here:

Data quality means to consume data. It is depend on size of the dataset. (Wang, 1996)

I have choosen exploring Trump twitter data set rom followimng link:

https://www.kaggle.com/code/domdejonge/exploring-trump-s-tweets-over-the-years

Usually public dataset like social media tweets have lots of noises. lide it has duplication, null values ,
special charactors. In addition to that, the data quality problem of the dataset is mostly comes as inconsistence of data, 
accuracy.

The data quality problems can be four categories.
completeness, consistency, timeliness, validity, and uniqueness
Intrinsic data quality: Intrinsic dimensions include accuracy, objectivity, believability and reputation
Contextual: Inaccurate data in use
Represenational data quality: Focus on consistency od the data and interpritability
Accessibility: It is leveraging the access in the dadabes to have fully data for use. makesure no missing data in teh dataset.

Here is the strategy to clean the data to get better accuracy.
    Remove duplication
    Remove special characters that makes noise in the dataset
    Filer null values 
    fix structural errors
    hide missing data
    replace missing data with means values


Reference:

Wang, R. Y. (1996). Beyond accuracy: What data quality means to data consumers. Journal of Management Information Systems, 12(4), 5–34. https://doi.org/10.1080/07421222.1996.11518099


'''